## Pricing engines test - Anaytical vs FFT vs Montecarlo

This notebook runs a set of diagnostic tests comparing the Analytical, FFT and Montecarlo engines. It is intentionally verbose with prints and small plots to help debugging.

### Notes:
 - Run from the repository notebooks folder. The first cell will take you to the root folder of the project so the necessary modules can be imported

In [ ]:
import os

# --- Step 1: Define the target folder relative to the current folder ---
# '..' means go up one level. '..\\..' means go up two levels.
relative_path_up_two = os.path.join('..', '..') 

# --- Step 2: Navigate up two levels (this assumes the notebook's current 
#             folder is the working directory, which is typical in Jupyter)
try:
    # Change the working directory to the target path
    os.chdir(relative_path_up_two)
    
    # Print the new directory for confirmation
    print(f"Successfully changed directory up two levels to: \n{os.getcwd()}")
    
except FileNotFoundError:
    print("Error: Could not navigate up two levels. Check your starting path.")


Successfully changed directory up two levels to: 
c:\Users\ramon\OneDrive\Desktop\Python projects\OptionPricingPY


In [ ]:
from src.engines import *
from src.products import *
from src.models import *
from src.greeks import *

import numpy as np
import pandas as pd

# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("EJEMPLO 1: Black-Scholes con diferentes engines")
    print("="*70)
    
    expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=30)
    # Crear producto
    option = EuropeanOption(
        S=100,
        K=100,
        expiry_date=expiry_date,  # días
        option_type='call',
        qty=1
    )
    
    # Crear modelo
    bs_model = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)
    
    # Pricing con motor analítico
    analytical_engine = AnalyticalEngine()
    price_analytical = analytical_engine.calculate_price(option, bs_model)
    print(f"\nPrecio Analítico: {price_analytical:.4f}")
    
    # Pricing con FFT
    fft_engine = FFTEngine()
    price_fft = fft_engine.calculate_price(option, bs_model)
    print(f"Precio FFT: {price_fft:.4f}")
    
    # Pricing con Monte Carlo
    mc_engine = MonteCarloEngine(n_paths=100000, seed=42)
    price_mc = mc_engine.calculate_price(option, bs_model)
    print(f"Precio Monte Carlo: {price_mc:.4f}")
    
    print("\n" + "="*70)
    print("EJEMPLO 2: Heston con FFT vs Monte Carlo")
    print("="*70)
    
    # Parámetros del modelo Heston
    heston_model = HestonModel(
        kappa=2.0,      # velocidad de reversión a la media
        theta=0.04,     # varianza de largo plazo
        sigma=0.3,      # volatilidad de la volatilidad
        rho=-0.7,       # correlación
        v0=0.04,        # varianza inicial
        r=0.05,
        q=0.02
    )
    
    price_heston_fft = fft_engine.calculate_price(option, heston_model)
    price_heston_mc = mc_engine.calculate_price(option, heston_model)
    
    print(f"\nPrecio Heston (FFT): {price_heston_fft:.4f}")
    print(f"Precio Heston (Monte Carlo): {price_heston_mc:.4f}")
    print(f"Diferencia: {abs(price_heston_fft - price_heston_mc):.4f}")
    
    print("\n" + "="*70)
    print("EJEMPLO 3: Comparación múltiples strikes")
    print("="*70)
    
    # Parámetros similares a tu ejemplo
    S0 = 100.0
    r = 0.01
    q = 0.35
    T = 365  # 1 año en días
    vol = 0.2
    
    # Heston con parámetros que reproducen Black-Scholes
    kappa = 0
    theta = (1.0*vol)**2
    sigma_h = 0.00001
    rho = 0.2
    v0 = (1.0*vol)**2
    
    bs_model_test = BlackScholesModel(sigma=vol, r=r, q=q)
    heston_model_test = HestonModel(
        kappa=kappa, theta=theta, sigma=sigma_h,
        rho=rho, v0=v0, r=r, q=q
    )
    
    # Array de strikes
    strikes = np.linspace(80, 120, 10)
    
    print(f"\nStrike | Analítico BS | Monte Carlo BS | FFT Heston | Monte Carlo Heston")
    print("-" * 72)
    
    mc_engine_test = MonteCarloEngine(n_paths=100000, n_steps=252, seed=42)
    
    prices_bs_analytical = []
    prices_bs_mc = []
    prices_heston_mc = []
    prices_heston_fft = []

    for K in strikes:
        expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=T)
        opt = EuropeanOption(S=S0, K=K, expiry_date=expiry_date, option_type='call', qty=1)
        
        price_bs_analytical = analytical_engine.calculate_price(opt, bs_model_test)
        price_bs_mc = mc_engine_test.calculate_price(opt, bs_model_test)
        price_heston_mc = mc_engine_test.calculate_price(opt, heston_model_test)
        price_heston_fft = fft_engine.calculate_price(opt, heston_model_test)

        prices_bs_analytical.append(price_bs_analytical)
        prices_bs_mc.append(price_bs_mc)
        prices_heston_mc.append(price_heston_mc)
        prices_heston_fft.append(price_heston_fft)

        print(f"{K:6.1f} | {price_bs_analytical:12.6f} | {price_bs_mc:14.6f} | {price_heston_fft:11.6f}| {price_heston_mc:18.6f}")
    
    # Calcular error L2 

    l2_error_analytical_fft = np.linalg.norm(
        np.array(prices_bs_analytical) - np.array(prices_heston_fft)
    )
    l2_error_fft_mc = np.linalg.norm(
        np.array(prices_bs_mc) - np.array(prices_heston_mc)
    )
    
    print(f"\nL2 norm difference (BS Analytical vs BS FFT): {l2_error_analytical_fft:.6f}")
    print(f"L2 norm difference (Heston FFT vs Heston Monte Carlo): {l2_error_fft_mc:.6f}")

EJEMPLO 1: Black-Scholes con diferentes engines

Precio Analítico: 2.4056
Precio FFT: 2.4056
Precio Monte Carlo: 2.4113

EJEMPLO 2: Heston con FFT vs Monte Carlo

Precio Heston (FFT): 2.3939
Precio Heston (Monte Carlo): 2.3938
Diferencia: 0.0002

EJEMPLO 3: Comparación múltiples strikes

Strike | Analítico BS | Monte Carlo BS | FFT Heston | Monte Carlo Heston
------------------------------------------------------------------------
  80.0 |     2.578401 |       2.592104 |    2.578411|           2.572518
  84.4 |     1.669193 |       1.677501 |    1.669205|           1.667204
  88.9 |     1.053815 |       1.057508 |    1.053824|           1.051047
  93.3 |     0.650601 |       0.652089 |    0.650609|           0.647074
  97.8 |     0.393810 |       0.393698 |    0.393815|           0.391982
 102.2 |     0.234278 |       0.232838 |    0.234280|           0.234210
 106.7 |     0.137279 |       0.135565 |    0.137284|           0.138752
 111.1 |     0.079392 |       0.077841 |    0.079392| 

In [ ]:
# sin cambios en tus engines/models/products
from src.valuation.OptionValuationContext import OptionValuationContext

context = OptionValuationContext(mc_engine)          # engine ya creado, p.e. MonteCarloEngine(...)
price = context.value_option(option, bs_model)      # equivalente a mc_engine.calculate_price(option, bs_model)

# batch (ejemplo con strikes)
products = [EuropeanOption(S=S0, K=K, expiry_date=expiry_date, option_type='call', qty=1) for K in strikes]
prices = context.value_options(products, heston_model_test)

In [ ]:
# ejemplo de uso: cambiar engine sin cambiar cliente
context = OptionValuationContext(AnalyticalEngine())
p1 = context.value_option(product, bs_model)

# cambiar a MonteCarlo para comparar
context.engine = MonteCarloEngine(n_paths=100000, seed=42)
p2 = context.value_option(product, bs_model)

# batch
prices = context.value_options(list_of_products, heston_model)

In [ ]:
from src.valuation.OptionValuationContext import OptionValuationContext
from src.engines.engines import MonteCarloEngine, AnalyticalEngine
from src.products.products import EuropeanOption
from src.models.models import BlackScholesModel

mc = MonteCarloEngine(n_paths=50000, seed=42)
ctx = OptionValuationContext(mc, cache_enabled=True, cache_maxsize=512, parallel=True, max_workers=4)

opt = EuropeanOption(S=100, K=100, T=30, option_type="call", qty=1)
bs = BlackScholesModel(sigma=0.2, r=0.01, q=0.0)

price = ctx.value_option(opt, bs)               # single
prices = ctx.value_options([opt]*10, bs)       # batch (parallel)

In [4]:
import os
import numpy as np
import time
import pandas as pd

from src.engines import *
from src.products import *
from src.models import *
from src.greeks import *

os.chdir(r"C:\Users\ramon\OneDrive\Desktop\Python projects\OptionPricingPY")

import logging
from src.valuation.OptionValuationContext import OptionValuationContext
import numpy as np

# Setup logging para ver los hooks
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

print("="*70)
print("DEMO: OptionValuationContext con logging, cache y paralelización")
print("="*70)

# ============================================================================
# PARTE 1: Single valuation con Context vs Raw Engine
# ============================================================================
print("\n" + "="*70)
print("PARTE 1: Single valuation - Context vs Raw Engine")
print("="*70)

expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=30)
option = EuropeanOption(S=100, K=100, expiry_date=expiry_date, option_type='call', qty=1)
bs_model = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

# Raw engine (sin context)
print("\n[RAW ENGINE] Analytical:")
analytical_engine = AnalyticalEngine()
price_raw = analytical_engine.calculate_price(option, bs_model)
print(f"  Precio: {price_raw:.6f}")

# Con context (sin cache)
print("\n[CONTEXT] Analytical (sin cache):")
ctx_analytical = OptionValuationContext(analytical_engine, cache_enabled=False)
price_ctx = ctx_analytical.value_option(option, bs_model)
print(f"  Precio: {price_ctx:.6f}")

# Verificar que son iguales
assert abs(price_raw - price_ctx) < 1e-10, f"Mismatch! {price_raw} vs {price_ctx}"
print(f"  ✓ Precios coinciden: {price_raw:.6f} == {price_ctx:.6f}")

# ============================================================================
# PARTE 2: Cache behavior
# ============================================================================
print("\n" + "="*70)
print("PARTE 2: Cache behavior (miss/hit)")
print("="*70)

ctx_cached = OptionValuationContext(analytical_engine, cache_enabled=True, cache_maxsize=128)

print("\n[CACHE] Primera llamada (MISS):")
p1 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p1:.6f}")

print("\n[CACHE] Segunda llamada con mismo producto/modelo (HIT):")
p2 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p2:.6f}")

assert p1 == p2, "Cache hit devolvió valor diferente"
print(f"  ✓ Cache funciona: {p1:.6f} == {p2:.6f}")

print("\n[CACHE] Tercera llamada con K diferente (MISS):")
expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=30)
option2 = EuropeanOption(S=100, K=110, expiry_date=expiry_date, option_type='call', qty=1)
p3 = ctx_cached.value_option(option2, bs_model)
print(f"  Precio: {p3:.6f}")
assert p3 != p1, "Cache no diferencia strikes"
print(f"  ✓ Cache diferencia strikes: {p1:.6f} != {p3:.6f}")

# ============================================================================
# PARTE 3: Batch valuation sin paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 3: Batch valuation (secuencial)")
print("="*70)

strikes = np.linspace(80, 120, 10)
expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=30)
products_batch = [EuropeanOption(S=100, K=K,expiry_date=expiry_date, option_type='call', qty=1) for K in strikes]

print(f"\nValuando {len(products_batch)} opciones (secuencial)...")
ctx_batch = OptionValuationContext(analytical_engine, cache_enabled=False, parallel=False)
prices_ctx_batch = ctx_batch.value_options(products_batch, bs_model)

print("\n[RAW ENGINE] Precios con loop:")
prices_raw_batch = [analytical_engine.calculate_price(p, bs_model) for p in products_batch]

print("\nComparación strikes:")
print(f"{'Strike':<10} {'Context':<15} {'Raw':<15} {'Diff':<15}")
print("-" * 55)
for K, p_ctx, p_raw in zip(strikes, prices_ctx_batch, prices_raw_batch):
    diff = abs(p_ctx - p_raw)
    print(f"{K:<10.2f} {p_ctx:<15.6f} {p_raw:<15.6f} {diff:<15.6e}")
    assert diff < 1e-10, f"Mismatch en K={K}"
print("✓ Todos los precios coinciden")

# ============================================================================
# PARTE 4: Batch valuation CON paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 4: Batch valuation (paralelizado)")
print("="*70)

mc_engine = MonteCarloEngine(n_paths=50000, seed=42)

print(f"\nValuando {len(products_batch)} opciones (PARALELO, max_workers=4)...")

# Progress callback para ver ejecución
def progress_cb(idx, price):
    if price is not None:
        print(f"  [Progress] Índice {idx} completado: precio={price:.6f}")

ctx_parallel = OptionValuationContext(
    mc_engine,
    cache_enabled=False,
    parallel=True,
    max_workers=4
)

prices_parallel = ctx_parallel.value_options(
    products_batch,
    bs_model,
    progress_callback=progress_cb
)

print(f"\n✓ {len(prices_parallel)} precios computados")
print(f"  Rango: [{min(prices_parallel):.6f}, {max(prices_parallel):.6f}]")

# ============================================================================
# PARTE 5: Engine switching (ventaja principal del Context)
# ============================================================================
print("\n" + "="*70)
print("PARTE 5: Engine switching sin cambiar cliente")
print("="*70)

T=365
expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=T)
option_test = EuropeanOption(S=100, K=100, expiry_date=expiry_date, option_type='call', qty=1)
bs_test = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

ctx = OptionValuationContext(AnalyticalEngine())

print("\n[ENGINE 1] Analytical:")
p_analytical = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_analytical:.6f}")

print("\n[ENGINE 2] FFT (cambio en tiempo de ejecución):")
ctx.engine = FFTEngine()
p_fft = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_fft:.6f}")
print(f"  Diferencia: {abs(p_analytical - p_fft):.6e}")

print("\n[ENGINE 3] Monte Carlo (cambio en tiempo de ejecución):")
ctx.engine = MonteCarloEngine(n_paths=100000, seed=42)
p_mc = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_mc:.6f}")
print(f"  Diferencia vs Analytical: {abs(p_analytical - p_mc):.6e}")

print("\n✓ Engine switching funciona correctamente sin cambiar código cliente")

# ============================================================================
# PARTE 6: Comparación Heston con múltiples engines
# ============================================================================
print("\n" + "="*70)
print("PARTE 6: Heston - FFT vs Monte Carlo vía Context")
print("="*70)

heston_model = HestonModel(
    kappa=2.0, theta=0.04, sigma=0.3, rho=-0.7, v0=0.04,
    r=0.05, q=0.02
)

T=365
expiry_date = pd.Timestamp.today().date() + pd.Timedelta(days=T)
option_heston = EuropeanOption(S=100, K=100, expiry_date=expiry_date, option_type='call', qty=1)

print("\n[CONTEXT + FFT] Heston:")
ctx_fft = OptionValuationContext(FFTEngine())
p_heston_fft = ctx_fft.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_fft:.6f}")

print("\n[CONTEXT + MC] Heston:")
ctx_mc = OptionValuationContext(MonteCarloEngine(n_paths=100000, seed=42))
p_heston_mc = ctx_mc.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_mc:.6f}")

diff_heston = abs(p_heston_fft - p_heston_mc)
print(f"\n  Diferencia: {diff_heston:.6f}")
print(f"  Diferencia relativa: {100 * diff_heston / p_heston_fft:.2f}%")

print("\n" + "="*70)
print("✓ DEMO COMPLETADA")
print("="*70)
#</VSCode.Cell>

DEBUG - Valuing product={'S': 100, 'K': 100, 'expiry_date': datetime.date(2026, 1, 14), 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 2.40562374407466
DEBUG - Valuing product={'S': 100, 'K': 100, 'expiry_date': datetime.date(2026, 1, 14), 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 2.40562374407466
DEBUG - Cache hit for key=90449b900dd64d71091faefbe01a74f3a3fa258d94ddf1efd848276ac34ca909
DEBUG - Valuing product={'S': 100, 'K': 110, 'expiry_date': datetime.date(2026, 1, 14), 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 0.13312693878012816
DEBUG - Valuing product={'S': 100, 'K': np.float64(80.0), 'expiry_date': datetime.date(2026, 1, 14), 'option_type': 'call', 'qty': 1} model={'sigma': 0.2, 'r': 0.05, 'q': 0.02} kwargs={}
DEBUG - Price computed: 20.163892661843278
DEBUG - Valuing product={'S': 1

DEMO: OptionValuationContext con logging, cache y paralelización

PARTE 1: Single valuation - Context vs Raw Engine

[RAW ENGINE] Analytical:
  Precio: 2.405624

[CONTEXT] Analytical (sin cache):
  Precio: 2.405624
  ✓ Precios coinciden: 2.405624 == 2.405624

PARTE 2: Cache behavior (miss/hit)

[CACHE] Primera llamada (MISS):
  Precio: 2.405624

[CACHE] Segunda llamada con mismo producto/modelo (HIT):
  Precio: 2.405624
  ✓ Cache funciona: 2.405624 == 2.405624

[CACHE] Tercera llamada con K diferente (MISS):
  Precio: 0.133127
  ✓ Cache diferencia strikes: 2.405624 != 0.133127

PARTE 3: Batch valuation (secuencial)

Valuando 10 opciones (secuencial)...

[RAW ENGINE] Precios con loop:

Comparación strikes:
Strike     Context         Raw             Diff           
-------------------------------------------------------
80.00      20.163893       20.163893       0.000000e+00   
84.44      15.739693       15.739693       0.000000e+00   
88.89      11.346537       11.346537       0.000000e

DEBUG - Price computed: 9.059506894705038
DEBUG - Valuing product={'S': 100, 'K': 100, 'expiry_date': datetime.date(2026, 12, 15), 'option_type': 'call', 'qty': 1} model={'kappa': 2.0, 'theta': 0.04, 'sigma': 0.3, 'rho': -0.7, 'v0': 0.04, 'r': 0.05, 'q': 0.02} kwargs={}


  Precio: 9.059507

[CONTEXT + MC] Heston:


DEBUG - Price computed: 9.067131290998292


  Precio: 9.067131

  Diferencia: 0.007624
  Diferencia relativa: 0.08%

✓ DEMO COMPLETADA


In [3]:
import logging
from src.core.orchestration.pricing_engine import OptionValuationContext
from src.engines.engines import AnalyticalEngine, MonteCarloEngine, FFTEngine
from src.products.products import EuropeanOption
from src.models.models import BlackScholesModel, HestonModel
import numpy as np

# Setup logging para ver los hooks
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

print("="*70)
print("DEMO: OptionValuationContext con logging, cache y paralelización")
print("="*70)

# ============================================================================
# PARTE 1: Single valuation con Context vs Raw Engine
# ============================================================================
print("\n" + "="*70)
print("PARTE 1: Single valuation - Context vs Raw Engine")
print("="*70)

option = EuropeanOption(S=100, K=100, T=30, option_type='call', qty=1)
bs_model = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

# Raw engine (sin context)
print("\n[RAW ENGINE] Analytical:")
analytical_engine = AnalyticalEngine()
price_raw = analytical_engine.calculate_price(option, bs_model)
print(f"  Precio: {price_raw:.6f}")

# Con context (sin cache)
print("\n[CONTEXT] Analytical (sin cache):")
ctx_analytical = OptionValuationContext(analytical_engine, cache_enabled=False)
price_ctx = ctx_analytical.value_option(option, bs_model)
print(f"  Precio: {price_ctx:.6f}")

# Verificar que son iguales
assert abs(price_raw - price_ctx) < 1e-10, f"Mismatch! {price_raw} vs {price_ctx}"
print(f"  ✓ Precios coinciden: {price_raw:.6f} == {price_ctx:.6f}")

# ============================================================================
# PARTE 2: Cache behavior
# ============================================================================
print("\n" + "="*70)
print("PARTE 2: Cache behavior (miss/hit)")
print("="*70)

ctx_cached = OptionValuationContext(analytical_engine, cache_enabled=True, cache_maxsize=128)

print("\n[CACHE] Primera llamada (MISS):")
p1 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p1:.6f}")

print("\n[CACHE] Segunda llamada con mismo producto/modelo (HIT):")
p2 = ctx_cached.value_option(option, bs_model)
print(f"  Precio: {p2:.6f}")

assert p1 == p2, "Cache hit devolvió valor diferente"
print(f"  ✓ Cache funciona: {p1:.6f} == {p2:.6f}")

print("\n[CACHE] Tercera llamada con K diferente (MISS):")
option2 = EuropeanOption(S=100, K=110, T=30, option_type='call', qty=1)
p3 = ctx_cached.value_option(option2, bs_model)
print(f"  Precio: {p3:.6f}")
assert p3 != p1, "Cache no diferencia strikes"
print(f"  ✓ Cache diferencia strikes: {p1:.6f} != {p3:.6f}")

# ============================================================================
# PARTE 3: Batch valuation sin paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 3: Batch valuation (secuencial)")
print("="*70)

strikes = np.linspace(80, 120, 10)
products_batch = [EuropeanOption(S=100, K=K, T=30, option_type='call', qty=1) for K in strikes]

print(f"\nValuando {len(products_batch)} opciones (secuencial)...")
ctx_batch = OptionValuationContext(analytical_engine, cache_enabled=False, parallel=False)
prices_ctx_batch = ctx_batch.value_options(products_batch, bs_model)

print("\n[RAW ENGINE] Precios con loop:")
prices_raw_batch = [analytical_engine.calculate_price(p, bs_model) for p in products_batch]

print("\nComparación strikes:")
print(f"{'Strike':<10} {'Context':<15} {'Raw':<15} {'Diff':<15}")
print("-" * 55)
for K, p_ctx, p_raw in zip(strikes, prices_ctx_batch, prices_raw_batch):
    diff = abs(p_ctx - p_raw)
    print(f"{K:<10.2f} {p_ctx:<15.6f} {p_raw:<15.6f} {diff:<15.6e}")
    assert diff < 1e-10, f"Mismatch en K={K}"
print("✓ Todos los precios coinciden")

# ============================================================================
# PARTE 4: Batch valuation CON paralelización
# ============================================================================
print("\n" + "="*70)
print("PARTE 4: Batch valuation (paralelizado)")
print("="*70)

mc_engine = MonteCarloEngine(n_paths=50000, seed=42)

print(f"\nValuando {len(products_batch)} opciones (PARALELO, max_workers=4)...")

# Progress callback para ver ejecución
def progress_cb(idx, price):
    if price is not None:
        print(f"  [Progress] Índice {idx} completado: precio={price:.6f}")

ctx_parallel = OptionValuationContext(
    mc_engine,
    cache_enabled=False,
    parallel=True,
    max_workers=4
)

prices_parallel = ctx_parallel.value_options(
    products_batch,
    bs_model,
    progress_callback=progress_cb
)

print(f"\n✓ {len(prices_parallel)} precios computados")
print(f"  Rango: [{min(prices_parallel):.6f}, {max(prices_parallel):.6f}]")

# ============================================================================
# PARTE 5: Engine switching (ventaja principal del Context)
# ============================================================================
print("\n" + "="*70)
print("PARTE 5: Engine switching sin cambiar cliente")
print("="*70)

option_test = EuropeanOption(S=100, K=100, T=365, option_type='call', qty=1)
bs_test = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)

ctx = OptionValuationContext(AnalyticalEngine())

print("\n[ENGINE 1] Analytical:")
p_analytical = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_analytical:.6f}")

print("\n[ENGINE 2] FFT (cambio en tiempo de ejecución):")
ctx.engine = FFTEngine()
p_fft = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_fft:.6f}")
print(f"  Diferencia: {abs(p_analytical - p_fft):.6e}")

print("\n[ENGINE 3] Monte Carlo (cambio en tiempo de ejecución):")
ctx.engine = MonteCarloEngine(n_paths=100000, seed=42)
p_mc = ctx.value_option(option_test, bs_test)
print(f"  Precio: {p_mc:.6f}")
print(f"  Diferencia vs Analytical: {abs(p_analytical - p_mc):.6e}")

print("\n✓ Engine switching funciona correctamente sin cambiar código cliente")

# ============================================================================
# PARTE 6: Comparación Heston con múltiples engines
# ============================================================================
print("\n" + "="*70)
print("PARTE 6: Heston - FFT vs Monte Carlo vía Context")
print("="*70)

heston_model = HestonModel(
    kappa=2.0, theta=0.04, sigma=0.3, rho=-0.7, v0=0.04,
    r=0.05, q=0.02
)

option_heston = EuropeanOption(S=100, K=100, T=30, option_type='call', qty=1)

print("\n[CONTEXT + FFT] Heston:")
ctx_fft = OptionValuationContext(FFTEngine())
p_heston_fft = ctx_fft.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_fft:.6f}")

print("\n[CONTEXT + MC] Heston:")
ctx_mc = OptionValuationContext(MonteCarloEngine(n_paths=100000, seed=42))
p_heston_mc = ctx_mc.value_option(option_heston, heston_model)
print(f"  Precio: {p_heston_mc:.6f}")

diff_heston = abs(p_heston_fft - p_heston_mc)
print(f"\n  Diferencia: {diff_heston:.6f}")
print(f"  Diferencia relativa: {100 * diff_heston / p_heston_fft:.2f}%")

print("\n" + "="*70)
print("✓ DEMO COMPLETADA")
print("="*70)

DEMO: OptionValuationContext con logging, cache y paralelización

PARTE 1: Single valuation - Context vs Raw Engine

[RAW ENGINE] Analytical:
  Precio: 2.405624

[CONTEXT] Analytical (sin cache):


TypeError: 'dict' object is not callable